In [ ]:
# Import necessary libraries
from bs4 import BeautifulSoup
import requests
import smtplib
import time
import datetime
import pandas as pd
import csv

In [1]:
# URL of the Amazon product page
URL = 'https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ'

# Headers for HTTP request to simulate a browser visit
# Use https://httpbin.org/get and adjust this section to your own computer
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Upgrade-Insecure-Requests": "1"
}

In [ ]:
# Function to scrape data
def scrape_data():
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    prettified_soup = BeautifulSoup(soup.prettify(), 'html.parser')

    title = prettified_soup.find(id='productTitle').get_text(strip=True)
    price_whole = prettified_soup.find('span', class_='a-price-whole').get_text(strip=True).replace('.', '')
    price_fraction = prettified_soup.find('span', class_='a-price-fraction').get_text(strip=True)
    price = f"{price_whole}.{price_fraction}"
    rating = prettified_soup.find('span', class_='a-icon-alt').get_text().split(' ')[0]
    review_count = ''.join(filter(str.isdigit, prettified_soup.find('span', id='acrCustomerReviewText').get_text(strip=True)))

    return [title, price, rating, review_count, datetime.date.today()]

In [ ]:
# Path to CSV file (ensure this is changed to your path)
csv_file_path = '/path/to/your/AmazonWebScraperDataset.csv'

In [ ]:
# Function to write data to CSV
def write_to_csv(data):
    with open(csv_file_path, 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

In [ ]:
# Function to send email alert
def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login('your-email@gmail.com', 'your-password')
    subject = "The Shirt you want is below $15! Now is your chance to buy!"
    body = ("This is the moment we have been waiting for. Now is your chance to pick up the shirt of your dreams. "
            "Don't mess it up! Link here: " + URL)
    msg = f"Subject: {subject}\n\n{body}"
    server.sendmail('your-email@gmail.com', 'your-email@gmail.com', msg)
    server.quit()

In [ ]:
# Check price and send email if under threshold
def check_price():
    data = scrape_data()
    write_to_csv(data)
    if float(data[1]) < 15:
        send_mail()

In [ ]:
# Scheduler to run the check daily
while True:
    check_price()
    time.sleep(86400)  # Sleep for 24h